# Front-tracking algorithm

## Introduction
 
 Validation made by: GB

 Report generated 04/03/2025

### Description of the algorithm

The purpose is to describe the algorithm at coarse grain. A description of the resolution steps is provided in FT-short.pdf see `share/doc/Multiphase/short-FT-description`.

## Build test cases

In [ ]:

import os

from trustutils import run

run.TRUST_parameters("1.9.6_beta")

In [ ]:
from trustutils import run

# Clean build folder
run.reset()
run.initBuildDirectory()


#prepare 
run.executeCommand("gmsh -2 pave/pave.geo -format msh2")
run.executeCommand("./pave/cree_multi_bulles.sh pave/deplacements.txt pave/init.lata")


In [ ]:

tc1=run.addCase(".","algorithm_ft.data",nbProcs=1)
tc2=run.addCase(".","algorithm_monop.data",nbProcs=1)

if run.isExtractingNR():
    tc1.substitute("fichier lata/post.lata","")
    tc2.substitute("fichier lata/monop.lata","")
# run.executeCommand("tar xzf sauv_lata.tgz",verbose=1)
run.printCases()

run.runCases() # Mettre prevent_concurrent si pre_run.data et changer "repr_file":"very_coarse dans dic_cas par "pre_run"

## Velocity solution
We solve for the momentum as: 
$$\frac{\partial \vec{u}}{\partial t} = \vec{g} $$
with $g=10 \vec{e}_z$ and $\vec{u}(t=0)=0$.

The solution is then a uniform velocity field:
$$\vec{u} (t) = 10 t\,\vec{e}_z$$

## Interface position:
Then, the interfaces (left at $x_l^0$ and right at $x_r^0$) are moving as:
$$x_s(t) = x_s^0+u (t) \times t \quad \mbox{for } s \in [l,r]$$

By substitution, we get:
$$x_s(t) = x_s^0+10 t^2 \quad \mbox{for } s \in [l,r]$$


## Analytical developments

In [ ]:
import os
os.chdir(os.path.join(run.BUILD_DIRECTORY))

In [ ]:
# lata_analyzer lata/post.lata asciiout writelata=ascii.lata
import LataLoader as ll
import numpy as np

fic = "lata/post.lata"
if (not os.path.isfile(fic)):
    raise Exception(f"File lata {fic} missing")

db = ll.LataLoader(fic)
if ('INTERFACES' not in db.GetMeshNames()):
    raise Exception(f"Missing INTERFACES in {fic}. Available: {db.GetMeshNames()}")

for fds in ['INDICATRICE_ELEM_DOM', 'OLD_INDICATRICE_ELEM_DOM', 'TEMPERATURE_0_ELEM_DOM', 'VELOCITY_FACES_DOM_dual', 'VELOCITY_FT_FACES_DOM_EXT_dual']:
    if (fds not in db.GetFieldNames()):
        raise Exception(f"Missing field {fds} in {fic}. Available: {db.GetFieldNames()}")
    
times=np.array(db.getTimes())
nt=db.GetNTimesteps()
xl = np.zeros(nt)
xr = np.zeros(nt)
vz = np.zeros(nt) # Velocity is uniform, we get the mean value.
vz_ext = np.zeros(nt) 
for j, t in enumerate(times):
    m=db.GetMesh('INTERFACES', j)
    zmin, zmax = m.getBoundingBox()[2]
    xl[j] = zmin
    xr[j] = zmax
    vz[j] = db.GetFieldDouble('VELOCITY_FACES_DOM_dual',j)[:,2].getWeightedAverageValue()[0]
    vz_ext[j] = db.GetFieldDouble('VELOCITY_FT_FACES_DOM_EXT_dual',j)[:,2].getWeightedAverageValue()[0]
    pass

print(f"Fields are: {db.GetFieldNames()}")

In [ ]:
def getNumPy(field):
    return np.array(field.getArray().getValues())

m=db.GetMesh('DOM',0)

# These are faces, so there is -1 elems:
ni=m.getCoordsAt(0).getNbOfElems()-1
nj=m.getCoordsAt(1).getNbOfElems()-1
nk=m.getCoordsAt(2).getNbOfElems()-1
nplane=ni*nj
i=0;j=0
I = np.zeros((nk,nt))
Iold = np.zeros((nk,nt))
T = np.zeros((nk,nt))
for it, t in enumerate(times):
    f = db.GetFieldDouble('INDICATRICE_ELEM_DOM',it)
    indic_1D=f[i+nj*j::nplane,:]
    I[:,it]=indic_1D.getArray().getValues()
    f = db.GetFieldDouble('OLD_INDICATRICE_ELEM_DOM',it)
    indold_1D=f[i+nj*j::nplane,:]
    Iold[:,it]=indold_1D.getArray().getValues()
    f = db.GetFieldDouble('TEMPERATURE_0_ELEM_DOM',it)
    Temp_1D=f[i+nj*j::nplane,:]
    T[:,it]=Temp_1D.getArray().getValues()
    del f
    

In [ ]:
import numpy as np

# Pour remplir les coordonnees
dom = db.GetMesh('DOM',0)
bb= dom.getBoundingBox()
xtmp=dom.getCoordsAt(2)
x=np.array(xtmp.getValues())
nx=len(x)

#nx = 24
#x=np.linspace(-0.7,1.1,nx+1)
xc=(x[1:]+x[:-1])/2.
#dt=0.06123724356957945
#nt=7
#times=dt*np.arange(nt)
g=10
u_ana=g*times

# The initial bubble is between 0 and 1
xl_ini = xl[0]; xr_ini = xr[0]
xl_ana = xl_ini+u_ana*times
xr_ana = xr_ini+u_ana*times
 
xl_ana_explicit = np.ones(nt)
xr_ana_explicit = np.ones(nt)
u_ana_explicit = np.ones(nt)
for j,t in enumerate(times) :
    if (j==0):
        #First step is initialize
        xl_ana_explicit[j] = xl_ini
        xr_ana_explicit[j] = xr_ini
        u_ana_explicit[j] = 0.
    else:
        # Euler expli:
        u_n = u_ana[j-1] # En explicit, on n'a pas encore le u du temps "n+1"
        dt = times[j]-times[j-1]
        xl_ana_explicit[j] = xl_ana_explicit[j-1]+u_n*dt
        xr_ana_explicit[j] = xr_ana_explicit[j-1]+u_n*dt
        u_ana_explicit[j] = u_ana_explicit[j-1]+g*dt
        
dx = (x[1:]-x[:-1]).mean()
print('dx=',dx)
def compute_indic(xl,xr):
    indic = np.ones((len(xc),len(times)))
    for j,t in enumerate(times) :
       for i,xx in enumerate(xc) :
           # The cell i is betwwen x[i] and x[i+1]
           # We check if not pure liquid to modify the indicatrice
           if ((xl[j]<x[i+1]) and (xr[j]>x[i])):
               if ((xl[j]<x[i]) and (xr[j]>x[i+1])):
                   # Pure gaz
                   indic[i,j] = 0
               elif ((xl[j]>x[i])):
                   # The left part of interface is in the cell:
                   indic[i,j] = (xl[j]-x[i])/dx
               else:
                   # The right part of interface is in the cell, we count the part between it and the right face:
                   indic[i,j] = (x[i+1]-xr[j])/dx
    return indic

# The phase-indicator based on the true markers position:
indic = compute_indic(xl,xr)

# Theoric indicator with an Euler explicit scheme
indic_ana_expli = compute_indic(xl_ana_explicit,xr_ana_explicit)
delta_indic_expli=indic-indic_ana_expli

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(14,5))
for i in range(4):
    plt.subplot(141+i)
    plt.title(f"ite {i} ; t={times[i]:,.4g}s")
    plt.plot(xc,indic_ana_expli[:,i], label='Ana. expli')
    plt.plot(xc,I[:,i], 'ro',label='I')
    plt.plot(xc,Iold[:,i], 'gx',label='Iold')
    plt.legend(loc=0)

print("Difference max between theoretical indicator (with exact velocity) and Euler one (with Euler expli. velocity)")
for i in range(len(times)): 
    print(f"error at it {i} : {np.max(np.abs(delta_indic_expli[:,i]))}")


delta_indic_expli_vs_lata=indic_ana_expli-I
print("Difference max between Analytical Euler indicator (based on Euler velocity) and the calculation one (in the lata)")
for i in range(len(times)): 
    print(f"error at it {i} : {np.max(np.abs(delta_indic_expli_vs_lata[:,i]))}")

On the indicator function, both 'Iold' and 'I' read from the lata are initialised identically (at step 0).
Then, at step 1, they remain identical (because $u^{0}=0$ due to the explicit scheme, so no displacement yet). 
At second step ($it=2$), only the new has been updated.
The same conclusion can be observed on probes below. Interface has moved by exactly one cell between step 2 and 3.

In [ ]:
from trustutils import plot
import numpy as np
import math

# To plot the variables of a segment of probes at a given time (by default, at the last time step), 
nY=4
Graph=plot.Graph("Indicator probes",size=[10,7], subtitle="xx", nY=4)

for i in range(nY):
    Graph.addPlot(i,f"ite {i} ; t={times[i]:,.4g}s")
    Graph.add(xc,indic_ana_expli[:,i],label=r"ana", color='k', marker='o')
    Graph.addSegment("algorithm_ft_PP_I.son",label=f"I -- it={i}",color="r",marker="-",compo=0,time=times[i]) 
    Graph.addSegment("algorithm_ft_PP_IOLD.son",label=f"Iold -- it={i}",color="r",marker="+",compo=0,time=times[i],markersize=24) 
    Graph.add(xc,I[:,i], 'bx',label='LATA',markersize=24)

Graph=plot.Graph("Indicator probes",size=[10,7], subtitle="xx", nY=4)
print(times)
for i in [4,5,6]:
    Graph.addPlot(i-4,f"ite {i} ; t={times[i]:,.4g}s")
    Graph.add(xc,indic_ana_expli[:,i],label=r"ana", color='k', marker='o')
    Graph.addSegment("algorithm_ft_PP_I.son",label=f"I -- it={i}",color="r",marker="-",compo=0,time=times[i]) 
    Graph.addSegment("algorithm_ft_PP_IOLD.son",label=f"Iold -- it={i}",color="r",marker="x",compo=0,time=times[i],markersize=24) 
    Graph.add(xc,I[:,i], 'bx',label='LATA',markersize=24)

In [ ]:
from trustutils import plot
import numpy as np
import math

# To plot the variables of a segment of probes at a given time (by default, at the last time step), 
Graph=plot.Graph("Temporal velocity evolution",size=[8,7], subtitle="Velocity", nY=2)
Graph.addPoint("algorithm_ft_PP_VZ.son",label="SONDE",color="r",marker="-",compo=0) # How to select the point on the probe?
#Graph.addPlot([0,1])
Graph.add(times,vz,label=r"LATA", color='k', marker='o')
Graph.add(times,vz_ext,label=r"LATA:V_EXT", color='b', marker='+')
Graph.add(times,u_ana,label=r"Analytical", color='k', marker='--')
Graph.add(times,u_ana_explicit,label=r"Ana. (Eul. expli)", color='k', marker='x')
Graph.addPoint("algorithm_ft_PP_VZEXT.son",label="SONDE:V_EXT",color="r",marker="-",compo=0)

Graph.label("time [s]","velocity [m/s]")

Graph.addPlot(1,"Error on velocity")
# it is possible to add a function Y=f(X,Y) in addSegment and addPoint (can allow to calculate for example an error)
def substraction(x,y):
    return y - 10.*x

Graph.addPoint("algorithm_ft_PP_VZ.son",compo=0,label=r"SONDE: $\delta v_z = v_z-v_z^{ana}$",func=substraction,color="r",marker="-")
Graph.add(times,(vz-u_ana),label=r"LATA: $\delta v_z = v_z-v_z^{ana}$", color='k', marker='o')
Graph.label("time [s]",r"velocity difference $\delta u$ [m/s]")

The velocity read from the lata field is in agreement with theory. We have an Euler explicit scheme and velocity increase linearly in time, so it can capture it properly.
Beware, the extended velocity (that is post_preocessed) on the extended domain is lagging one timestep behind (because its computation preceeds euler_explicit_update: 

`redistribute_from_splitting_ft_faces_[dir].redistribute_add(terme_source_interfaces_ft_[dir], d_velocity_[dir]);`

In [ ]:
from trustutils import plot
import numpy as np
import math

# To plot the variables of a segment of probes at a given time (by default, at the last time step), 
Graph=plot.Graph("Interface position",size=[8,7], subtitle="Position", nY=2)
Graph.add(times,xl_ana/dx,label=r"Analytical left $x_l$", color='k')
Graph.add(times,xl_ana_explicit/dx,label=r"Theo. Expli. left $x_l$", color='k', marker="-.+")

Graph.add(times,xl/dx,label=r"FT-mesh left $x_l$", color='k', marker='o')
Graph.add(times,xr_ana/dx,label=r"Analytical right $x_r$", color='r', marker='--')
Graph.add(times,xr_ana_explicit/dx,label=r"Theo. Expli. right $x_l$", color='r', marker="-.+")
Graph.add(times,xr/dx,label=r"FT-mesh right $x_r$", marker='x', color='r')
Graph.label("time [s]",r"Interface $x/\Delta x$ [-]")

Graph.addPlot(1,"Error on position")
Graph.add(times,(xl-xl_ana)/dx,label=r"Left $\delta x_l = x_l-x_l^{ana}$", color='k', marker='-o')
Graph.add(times,(xl-xl_ana_explicit)/dx,label=r"Left $\delta x_l^{expli} = x_l-x_l^{expli}$", color='k', marker='-.+')
Graph.add(times,(xr-xr_ana)/dx,label=r"Right $\delta x_r = x_r-x_r^{ana}$", color='r', marker='--x')
Graph.add(times,(xr-xr_ana_explicit)/dx,label=r"Right $\delta x_r^{expli} = x_r-x_r^{expli}$", color='r', marker='-.+')
Graph.label("time [s]",r"position difference $\delta x/\Delta x$ [-]")

## Interface transport

Regarding the interface position, analytical development proves that the interface displacement should evolve in $\propto t^2$. 
However, due to the explicit scheme, velocity to displace the marker is taken at the previous time ($n-1$). Thus, a theory based on Euler explicit discretisation gives a different position from the purely analytical solution. A good agreement with this theory is obtained on markers positions.

## Indicatrice and Temperature evolutions
Indicatrice is purely described by the markers here. It moves with their velocity. If the markers displacement were exact, then the position would be 'I'. However, due to Euler explicit scheme, we can compute another indicator 'II'. On the left Figure, this 'Ana. II' is shown and in good agreement. 
On the right, the figure shows that the code solution is in agreement with 'Ana II', as expected, and it illustrates the difference with 'Ana I'.

In [ ]:
from matplotlib.pyplot import cycler
import numpy as np
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import matplotlib.cm
import matplotlib.pyplot as plt

def get_cycle(cmap, N=None, use_index="auto"):
    if isinstance(cmap, str):
        if use_index == "auto":
            if cmap in ['Pastel1', 'Pastel2', 'Paired', 'Accent',
                        'Dark2', 'Set1', 'Set2', 'Set3',
                        'tab10', 'tab20', 'tab20b', 'tab20c']:
                use_index=True
            else:
                use_index=False
        cmap = plt.get_cmap(cmap)
    if not N:
        N = cmap.N
    if use_index=="auto":
        if cmap.N > 100:
            use_index=False
        elif isinstance(cmap, LinearSegmentedColormap):
            use_index=False
        elif isinstance(cmap, ListedColormap):
            use_index=True
    if use_index:
        ind = np.arange(int(N)) % cmap.N
        return cycler("color",cmap(ind))
    else:
        colors = cmap(np.linspace(0,1,N))
        return cycler("color",colors)

In [ ]:
dxx=dx
#TODO : HACK, I don't know how to scale the abscissa of the plot of a sonde
dxx=1

Graph=plot.Graph("Phase Indicator Function",size=[8,7], subtitle="Indicator", nY=2)
import matplotlib.pyplot as plt

Ncolors = 8
plt.rcParams["axes.prop_cycle"] = get_cycle("viridis", Ncolors)
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']
for j,t in enumerate(times):
    Graph.add(xc/dxx,indic_ana_expli[:,j],label=r"Ana II. t="+f"{t:,.4g}s", marker='-', color=colors[j])
    lab=None
    if (j==0): lab=r"LATA:I"
    Graph.add(xc/dxx,I[:,j],label=lab, marker='o', color=colors[j])
    if (j==0): lab=r"SONDE:I"
    Graph.addSegment("algorithm_ft_PP_I.son",label=lab,color=colors[j],marker="--x", markersize=12,compo=0,time=t)

Graph.label(r"$x_c/\Delta x$ [-]","Indicator [-]")
j=3
t=times[j]
Graph.addPlot(1,f"At {j}rd step, t={t:,.4g}")
Graph.add(xc/dxx,indic[:,j],label=r"Ana. I t="+f"{t:,.4g}s", marker='-', color=colors[j])
Graph.add(xc/dxx,indic_ana_expli[:,j],label=r"Ana II Expli", marker=":+", markersize=12, color=colors[j])
Graph.add(xc/dxx,I[:,j],label="LATA:I", marker='o', color=colors[j])

# To plot the variables of a segment of probes at a given time (by default, at the last time step), 
Graph.addSegment("algorithm_ft_PP_I.son",label="SONDE:I",color=colors[j],marker="--x", markersize=12,compo=0,time=times[j])
Graph.label(r"$x_c/\Delta x$ [-]","Indicator [-]")

# Temperature
The temperature is initialised as the indicator, and should remain equal to 'Ana II', as expected from an Euler-Explicit scheme. However, due to the convection scheme (instead of a pure transport of I obtained from the markers), the temperature is unbounded with this convection scheme. 

In [ ]:
from trustutils import plot
import numpy as np

Graph=plot.Graph("Temperature",size=[12,5], subtitle="Temperature", nX=3,nY=3)
for j,t in enumerate(times):
    ii, ij = int(j/3), j%3
    Graph.addPlot((ii,ij),f"At {j}rd step, t={t:,.4g}")
    Graph.add(xc/dxx,indic_ana_expli[:,j],label=r"Ana II. t="+f"{t:,.4g}s", marker='-', color=colors[j])
    lab=None
    if (j==0): lab=r"LATA:T"
    Graph.add(xc/dxx,T[:,j],label=lab, marker='o', color=colors[j])
    if (j==0): lab=r"SONDE:T"
    Graph.addSegment("algorithm_ft_PP_T.son",label=lab,color=colors[j],marker="--x", markersize=12,compo=0,time=t)
    Graph.label(r"Coordinate $x_c/\Delta x$ [-]","Temperature [-]")

Graph=plot.Graph("Temperature",size=[12,5], subtitle="Temperature", nX=1,nY=1)
j=3
t=times[j]
Graph.addPlot(0,f"At {j}rd step, t={t:,.4g}")
#Graph.add(xc/dxx,indic[:,j],label=r"Ana. I t="+f"{t:,.4g}s", marker='-', color=colors[j])
Graph.add(xc/dxx,indic_ana_expli[:,j],label=r"Ana II Expli", marker=":+", markersize=12, color=colors[j])
Graph.add(xc/dxx,T[:,j],label="LATA:T", marker='o', color=colors[j])

# To plot the variables of a segment of probes at a given time (by default, at the last time step), 
Graph.addSegment("algorithm_ft_PP_T.son",label="SONDE:T",color=colors[j],marker="--x", markersize=12,compo=0,time=times[j])
Graph.label(r"$x_c/\Delta x$ [-]","Temperature [-]")

# Method to frame the plot.
# All parameters are not mandatory:
Graph.visu(xmin=0.25,xmax=1,ymin=-0.7,ymax=1.1)

## Computer Performance

In [ ]:
run.tablePerf()